#### This notebook shows how to read the fastMRI dataset and apply some simple transformations to the data.

In [ ]:
%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

**Each file corresponds to one MRI scan and contains the k-space data, ground truth, and some meta data related to the scan**

# Masked Testing Data

In [ ]:
dataset_dir = '../data/example_data/'

In [ ]:
test_data1 = dataset_dir + 'example_data1.h5'
test_data2 = dataset_dir + 'example_data2.h5'

hf1, hf2 = h5py.File(test_data1), h5py.File(test_data2)

In [ ]:
'Keys:', list(hf1.keys())

In [ ]:
'Attrs:', dict(hf1.attrs)

In multi-coil MRIs, k-space has the following shape:
(number of slices, number of coils, height, width)

For single-coil MRIs, k-space has the following shape:
(number of slices, height, width)

**MRIs are acquired as 3D volumes, the first dimension is the number of 2D slices**

In [ ]:
volume_kspace = hf1['kspace'][()]
mask = hf1['mask'][()]

In [ ]:
volume_kspace.dtype, mask.dtype

In [ ]:
volume_kspace.shape, mask.shape

Let's see what the absolute value of k-space looks like:

In [ ]:
def show_multiple_slices(data, slice_nums, cmap=None):
    fig = plt.figure()
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)

In [ ]:
def show_slice(data, cmap=None):
    fig = plt.figure()
    plt.imshow(data, cmap=cmap)

In [ ]:
show_multiple_slices(np.log(np.abs(volume_kspace) + 1e-9), [1,20,30]) #show the 1st, 20th, and 30th slices

The fastMRI repo contains some utlity functions to convert k-space into image space. These functions work on PyTorch Tensors. The to_tensor function can convert Numpy arrays to PyTorch Tensors.

In [ ]:
import fastmri
from fastmri.data import transforms as T

In [ ]:
slice_kspace = T.to_tensor(volume_kspace)             # Convert from numpy array to pytorch tensor

In [ ]:
slice_image = fastmri.ifft2c(slice_kspace)           # Apply Inverse Fourier Transform to get the complex image

In [ ]:
# Compute absolute value to get a real image
slice_image_abs = fastmri.complex_abs(slice_image)

In [ ]:
show_multiple_slices(slice_image_abs, [10,20,30], cmap='gray')

MR imaging is an indirect process, whereby cross-sectional images of the subject’s anatomy are
produced from frequency and phase measurements instead of direct, spatially-resolved measurements. As we can see, each image in the dataset focuses on a different snapshot of the subject's anatomy. These slices can be combined into the full image using the Root-Sum-of-Squares (RSS) transform.

In [ ]:
slice_image_rss = fastmri.rss(slice_image_abs, dim=0)

# Before Reconstruction

In [ ]:
plt.imshow(np.abs(slice_image_rss.numpy()), cmap='gray')

# After Reconstruction

In [ ]:
reconstructed = 'fastmri_examples/unet/output/reconstructions/file1000304.h5'
hf_r = h5py.File(reconstructed)

In [ ]:
print('Keys:', list(hf_r.keys()))
print('Attrs:', dict(hf_r.attrs))

In [ ]:
volume_kspace_r = hf_r['reconstruction'][()]
print(volume_kspace_r.dtype)
print(volume_kspace_r.shape)

In [ ]:
fig = plt.figure()
plt.imshow(np.log(np.abs(volume_kspace_r[0][0]) + 1e-9), cmap="gray")

#show_multiple_slices(np.log(np.abs(volume_kspace_r) + 1e-9)) #show the 10th, 20th, and 30th slices

# Simulating Under-Sampled Data

So far, we have been looking at fully-sampled data. We can simulate under-sampled data by creating a mask and applying it to k-space.

In [ ]:
from fastmri.data.subsample import RandomMaskFunc
mask_func = RandomMaskFunc(center_fractions=[0.04], accelerations=[8])  # Create the mask function object

In [ ]:
masked_kspace, mask, _ = T.apply_mask(slice_kspace, mask_func)   # Apply the mask to k-space

Let's see what the subsampled image looks like:

In [ ]:
sampled_image = fastmri.ifft2c(masked_kspace)           # Apply Inverse Fourier Transform to get the complex image
sampled_image_abs = fastmri.complex_abs(sampled_image)   # Compute absolute value to get a real image
sampled_image_rss = fastmri.rss(sampled_image_abs, dim=0)

In [ ]:
plt.imshow(np.abs(sampled_image_rss.numpy()), cmap='gray')